In [1]:
#Importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#Loading the dataset
data_path = "C:/Users/Mrinal Kalita/Python Projects/MAchine Translation/fra.txt"

In [3]:
#Vectorizing the data and storing them into a list
input_texts = []
target_texts = []

#Creating vocabulary of unique characters
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding="utf-8") as f:
    lines = f.read().split('\n')

for line in lines[:50000]: #TAking only the 50000 samples

    input_text,target_text,_= line.split("\t")
    target_text = "\t" + target_text + "\n" 
    # We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
max_sequence_length_input = max(len(txt) for txt in input_texts)
max_sequence_length_target = max(len(txt) for txt in target_texts)

print('Total number of samples:', len(input_texts))
print('Number of unique input tokens:', len(input_characters))
print('Number of unique target tokens:', len(target_characters))
print('Max sequence length for inputs:', max_sequence_length_input)
print('Max sequence length for target:', max_sequence_length_target)

Total number of samples: 50000
Number of unique input tokens: 78
Number of unique target tokens: 103
Max sequence length for inputs: 21
Max sequence length for target: 67


### Tokenizing and Vectorizing the texts

In [5]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [6]:
# Creating array of zeros with mentioned shape
encoder_input_data = np.zeros((len(input_texts), max_sequence_length_input, len(input_characters)),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_sequence_length_target, len(target_characters)),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_sequence_length_target, len(target_characters)),dtype='float32')

In [7]:
#Populating the araray
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] =1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0 
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i,t-1,target_token_index[char]] =1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

### Building the model (Using Encoder and Decoder Architecture)

In [8]:
from attention import AttentionLayer
from tensorflow.keras.layers import Concatenate

In [9]:
#Defining the encoder
import tensorflow as tf
encoder_input = tf.keras.Input(shape=(None, len(input_characters)))
encoder = tf.keras.layers.LSTM(256,return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_input)

encoder_state = [state_h, state_c]

#defining the decoder
decoder_input = tf.keras.Input(shape=(None, len(target_characters)))
decoder = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_output, _,_ = decoder(decoder_input,initial_state=encoder_state)

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_output])
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_output, attn_out])

decoder_dense = tf.keras.layers.Dense(len(target_characters),activation ='softmax')
decoder_output = decoder_dense(decoder_concat_input)

model = tf.keras.Model([encoder_input, decoder_input],decoder_output)

In [10]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 78)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 103)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  343040      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  368640      input_2[0][0]                    
                                                                 lstm[0][1]            

### Training the model

In [11]:
#defining the model checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("model-{val_loss:.2f}.h5", monitor="val_loss", verbose=1,mode='min', save_best_only=True, save_weights_only=True)
stop = EarlyStopping(monitor="val_loss", patience=15, mode='min')
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.125, patience=6, min_lr=1e-6, verbose=1, mode='min')

In [12]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ["accuracy"])
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=32,
          epochs=50,
          validation_split=0.2,callbacks=[checkpoint,stop,reduce_lr])

Epoch 1/50
1250/1250 [==============================] - ETA: 0s - loss: 0.7774 - accuracy: 0.7873
Epoch 00001: val_loss improved from inf to 0.71267, saving model to model-0.71.h5
1250/1250 [==============================] - 235s 188ms/step - loss: 0.7774 - accuracy: 0.7873 - val_loss: 0.7127 - val_accuracy: 0.7912
Epoch 2/50
1250/1250 [==============================] - ETA: 0s - loss: 0.5205 - accuracy: 0.8457
Epoch 00002: val_loss improved from 0.71267 to 0.60015, saving model to model-0.60.h5
1250/1250 [==============================] - 252s 202ms/step - loss: 0.5205 - accuracy: 0.8457 - val_loss: 0.6002 - val_accuracy: 0.8192
Epoch 3/50
1250/1250 [==============================] - ETA: 0s - loss: 0.4416 - accuracy: 0.8677
Epoch 00003: val_loss improved from 0.60015 to 0.52737, saving model to model-0.53.h5
1250/1250 [==============================] - 269s 215ms/step - loss: 0.4416 - accuracy: 0.8677 - val_loss: 0.5274 - val_accuracy: 0.8419
Epoch 4/50
1250/1250 [===================

In [13]:
#Saving the mmodel
model.save('model.h5')

In [14]:
model_json = model.to_json()
with open("NMT_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("NMT_model_weight.h5")
print("Saved model to disk")

Saved model to disk


In [15]:
# loading the model architecture and asigning the weights
json_file = open('NMT_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = tf.keras.models.model_from_json(loaded_model_json, custom_objects={'AttentionLayer': AttentionLayer})
# load weights into new model
model_loaded.load_weights("NMT_model_weight.h5")

### Inferece Stage

In [16]:
model.layers

In [17]:
# Manually deriving the decoder states and decoder outputs so that we can reuse decoder outputs as inputs for next step
encoder_input = model_loaded.input[0]  #loading encoder_inputs
encoder_output, state_h, state_c = model_loaded.layers[2].output
encoder_model = tf.keras.Model(encoder_input, [encoder_output, state_h, state_c])
#print(ecoder_output.shape)
decoder_state_input_h = tf.keras.Input(shape=(256,))
decoder_state_input_c = tf.keras.Input(shape=(256,))
decoder_hidden_state_input = tf.keras.Input(shape=(max_sequence_length_input,256))

decoder_input = model_loaded.layers[1].output
decoder = model_loaded.layers[3]
decoder_output, state_h, state_c = decoder(decoder_input, initial_state=[decoder_state_input_h, decoder_state_input_c])

attn_layer = model_loaded.layers[4]
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_output])

concate = model_loaded.layers[5]
decoder_inf_concat = concate([decoder_output, attn_out_inf])

decoder_dense = model_loaded.layers[6]
decoder_state = [state_h, state_c]
decoder_output = decoder_dense(decoder_inf_concat)

decoder_model = tf.keras.Model([decoder_input]+[decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],[decoder_output] + decoder_state)

In [18]:
# Mapping between integers or indexes and characters.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [19]:
#decoding the output sentence from tokenized data
def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    #generating empty target sequence of length 1
    target_seq = np.zeros((1,1,len(target_characters)))
    #Adding the start character in the target sequence
    target_seq[0,0,target_token_index["\t"]] =1.0

    stop_con = False
    decoded_sentence = ""

    while not stop_con:
        decoder_output, state_h, state_c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        # Sample a token
        sampled_token_index = np.argmax(decoder_output[0,-1,:])
        sample_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sample_char

    
        # Exit condition: either hit max length
        # or find stop character.
        if sample_char == "\n" or len(decoded_sentence) > max_sequence_length_target:
            stop_con = True
    
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1,len(target_characters)))
        target_seq[0, 0, sampled_token_index] = 1.0
    
        # Update states
        e_h, e_c = state_h, state_c

    return decoded_sentence

In [20]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !



In [21]:
text = "Go."
input_data = np.zeros((1, max_sequence_length_input, len(input_characters)),dtype='float32')
for t , char in enumerate(text):
    input_data[0,t, input_token_index[char]] = 1.0
input_data[0, t+1:, input_token_index[" "]] = 1.0

In [22]:
print(decode_sequence(input_data[0:1]))

Va !



In [23]:
#prediction for any random text
def eng_to_french(text):
    input_data = np.zeros((1,max_sequence_length_input, len(input_characters)),dtype='float32')
    for t, char in enumerate(text):
        input_data[0,t,input_token_index[char]] = 1.0
    input_data[0, t+1:, input_token_index[" "]] =1.0
    return print(decode_sequence(input_data[0:1]))

In [28]:
eng_to_french("Tea")

Il n'est pas un bas.

